In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from ipywidgets import interactive, fixed
pd.set_option('display.max_columns', None)

In [9]:
data = pd.read_csv("jupyter_measurements.csv")
data.head(12)

,Photo,UNIX,Date,Time,Io,Io_aligned,Europa,Europa_aligned,Ganymede,Ganymede_aligned,Callisto,Callisto_aligned
0,1,1286335254,06.10.2010,3:20:54,344.557,45.803,271.129,-27.625,NaN,NaN,533.380,234.626
1,2,1286335393,06.10.2010,3:23:13,344.140,45.386,270.938,-27.816,NaN,NaN,533.024,234.270
2,3,1286339042,06.10.2010,4:24:02,340.198,41.444,277.521,-21.233,NaN,NaN,533.468,234.714
3,4,1286341981,06.10.2010,5:13:01,335.921,37.167,282.817,-15.937,282.393,-16.361,533.236,234.482
4,5,1286342160,06.10.2010,5:16:00,335.598,36.844,282.571,-16.183,282.236,-16.518,533.197,234.443
5,6,1286345191,06.10.2010,6:06:31,335.522,36.768,282.849,-15.905,282.129,-16.625,533.004,234.250
6,7,1286345311,06.10.2010,6:08:31,330.781,32.027,288.030,-10.724,278.198,-20.556,533.361,234.607
7,8,1286348690,06.10.2010,7:04:50,325.071,26.317,NaN,NaN,272.874,-25.880,533.709,234.955
8,9,1286427002,07.10.2010,4:50:02,276.919,-21.835,381.921,83.167,187.423,-111.331,514.312,215.558
9,10,1286432162,07.10.2010,6:16:02,287.593,-11.161,381.309,82.555,183.659,-115.095,511.952,213.198


In [10]:
y_io = data["Io_aligned"]
y_europa = data["Europa_aligned"].fillna(method="ffill")
y_ganymede = data["Ganymede_aligned"].fillna(0)
y_callisto = data["Callisto_aligned"]
x_unix = data["UNIX"]

# accepted rotation period in days
val_io = 1.762732
val_europa = 3.525463
val_ganymede = 7.155588
val_callisto = 16.690440

# accepeted distance of satellites in km
dist_io = 421800
dist_europa = 671100
dist_ganymede = 1070400
dist_callisto = 1882700

In [73]:
def plot_moon(period=0.0, amp=0.0, shift=0.0, moon="Callisto_aligned"):
    start = data["UNIX"][0]
    stop = data["UNIX"][12]
    sns.scatterplot(x=data["UNIX"], y=data[moon])
    sns.set(rc={'figure.figsize':(10,10)})
    n_samples = 100
    w = 2 * np.pi /period
    x = np.linspace(start=start, stop=stop, num=n_samples)
    y = amp * np.sin(w*x+shift)
    plt.plot(x, y, "r-")
    # plt.figure(figsize=(7, 7))
    plt.legend(["Data", "Fitted curve"], fontsize="large", loc="upper right")
    plt.show()
    return x,y

In [74]:
def find_error(period, acc_val):
    days = period/(60*60*24)
    err = abs(days/acc_val-1)*100
    return err

In [75]:
interactive(plot_moon, period=(1400000, 1500000), amp=(1, 400), shift=(1, 2*np.pi), acc_val=val_callisto, moon="Callisto_aligned")

interactive(children=(IntSlider(value=1400000, description='period', max=1500000, min=1400000), IntSlider(valu…

In [76]:
interactive(plot_moon, period=(145000, 155000), amp=(1, 400), shift=(1, 3*np.pi), moon="Io_aligned")

interactive(children=(IntSlider(value=145000, description='period', max=155000, min=145000), IntSlider(value=1…

In [19]:
def find_error_distance(exp_val, acc_val):
    err = abs(exp_val/acc_val-1)*100
    return err

In [22]:
print(f"Io distance error{find_error_distance(dist_io, find_distance(e_dist_io)): .2f}%.")
print(f"Europa distance error{find_error_distance(dist_europa, find_distance(e_dist_europa)): .2f}%.")
print(f"Ganymede distance error{find_error_distance(dist_ganymede, find_distance(e_dist_ganymede)): .2f}%.")
print(f"Callisto distance error{find_error_distance(dist_callisto, find_distance(e_dist_callisto)): .2f}%.")

Io distance error 1.43%.
Europa distance error 1.10%.
Ganymede distance error 0.64%.
Callisto distance error 1.04%.


In [23]:
print(f"Io period error{find_error(e_val_io, val_io): .2f}%.")
print(f"Europa period error{find_error(e_val_europa, val_europa): .2f}%.")
print(f"Ganymede period error{find_error(e_val_ganymede, val_ganymede): .2f}%.")
print(f"Callisto period error{find_error(e_val_callisto, val_callisto): .2f}%.")

Io period error 0.01%.
Europa period error 0.04%.
Ganymede period error 0.07%.
Callisto period error 0.04%.


In [38]:
def find_mass(a, T):
    # convert period to seconds and radius to meters
    # T = T*24*60*60
    a = a*1000
    # Gravitational constant
    G = 6.673e-11
    
    m = ((a**3)*4*(np.pi**2)/((T**2*G)))
    return m

In [43]:
find_mass(a=find_distance(e_dist_callisto), T=e_val_callisto)

1.842166929776368e+27

In [46]:
jupiter_mass = 1.898e27

find_error_distance(find_mass(a=find_distance(e_dist_io), T=e_val_io), jupiter_mass)

1.8192730426752735

In [52]:
d = {'Satellites': ["Io", "Europa", "Ganymede", "Callisto"],
     'Accepted T (s)': [val_io, val_europa, val_ganymede, val_callisto],
     'Measured T (s)': [e_val_io, e_val_europa, e_val_ganymede, e_val_callisto],
     'Error in T (%)': [find_error(e_val_io, val_io), find_error(e_val_europa, val_europa), find_error(e_val_ganymede, val_ganymede), find_error(e_val_callisto, val_callisto)],
     'Accepted d (km)': [dist_io, dist_europa, dist_ganymede, dist_callisto],
     'Measured d (km)': [find_distance(e_dist_io), find_distance(e_dist_europa), find_distance(e_dist_ganymede), find_distance(e_dist_callisto)],
     'Error in d (%)': [find_error_distance(dist_io, find_distance(e_dist_io)), find_error_distance(dist_europa, find_distance(e_dist_europa)), find_error_distance(dist_ganymede, find_distance(e_dist_ganymede)), find_error_distance(dist_callisto, find_distance(e_dist_callisto))],
     'Jupiter M (kg)': [find_mass(a=find_distance(e_dist_io), T=e_val_io), find_mass(a=find_distance(e_dist_europa), T=e_val_europa),find_mass(a=find_distance(e_dist_ganymede), T=e_val_ganymede), find_mass(a=find_distance(e_dist_callisto), T=e_val_callisto)],
     'Error in M (%)': [find_error_distance(find_mass(a=find_distance(e_dist_io), T=e_val_io), jupiter_mass), find_error_distance(find_mass(a=find_distance(e_dist_europa), T=e_val_europa), jupiter_mass), find_error_distance(find_mass(a=find_distance(e_dist_ganymede), T=e_val_ganymede), jupiter_mass), find_error_distance(find_mass(a=find_distance(e_dist_callisto), T=e_val_callisto), jupiter_mass)]
     }
df = pd.DataFrame(data = d)
df['Accepted T (s)'] = round(df['Accepted T (s)']*(60*60*24))
df['Measured d (km)'] = round(df['Measured d (km)'])
df.to_csv('results.csv',index=False)
df

,Satellites,Accepted T (s),Measured T (s),Error in T (%),Accepted d (km),Measured d (km),Error in d (%),Jupiter M (kg),Error in M (%)
0,Io,152300.0,152283,0.011192,421800,415857.0,1.429022,1.834716e+27,3.334264
1,Europa,304600.0,304710,0.036112,671100,663772.0,1.103957,1.863470e+27,1.819273
2,Ganymede,618243.0,617839,0.065315,1070400,1063635.0,0.636023,1.864944e+27,1.741605
3,Callisto,1442054.0,1441452,0.041747,1882700,1863361.0,1.037876,1.842167e+27,2.941679
